In [1]:
#!pip install pymongo

In [5]:
import pymongo
import pandas as pd
from datetime import datetime, timedelta
import networkx as nx
import numpy as np

In [6]:
client=pymongo.MongoClient('mongodb://127.0.0.1:27017/')
mydb = client['Practica3']
table2 = mydb["tabla_estaciones"]
table = mydb["tabla_rutas"]

# Limpieza e inserción de datos.

In [3]:
path = '/home/axleon/Documents/CDD/6°_Sem/BDNE/BDNE_Tareas/Proyecto_Mongo/data/'
names = ['2013-07.csv', '2013-08.csv', '2013-09.csv', '2013-10.csv', '2013-11.csv', '2013-12.csv', '2014-01.csv', '2014-02.csv']
all_data = []
for file in names:
    all_data.append(pd.read_csv(path + file)[['tripduration', 'starttime', 'stoptime', 'start station id','start station name', 'start station latitude', 'start station longitude', 'end station id', 'end station name', 'end station latitude', 'end station longitude']])

In [4]:
for df in all_data:
    df['starttime'] = pd.to_datetime(df['starttime'], format='%Y-%m-%d')

### Tabla estaciones

In [ ]:
information = mydb.tabla_estaciones

In [5]:
locali = pd.DataFrame()
for df in all_data:
    loc = df[['start station name','start station latitude','start station longitude']].drop_duplicates().reset_index(drop=True)
    locali = pd.concat([locali, loc]).drop_duplicates()

locali

,start station name,start station latitude,start station longitude
0,E 47 St & 2 Ave,40.753231,-73.970325
1,W 26 St & 10 Ave,40.749718,-74.002950
2,Lafayette St & E 8 St,40.730287,-73.990765
3,Forsyth St & Broome St,40.718939,-73.992663
4,University Pl & E 14 St,40.734927,-73.992005
...,...,...,...
325,Franklin Ave & Myrtle Ave,40.694528,-73.958089
326,Park Ave & St Edwards St,40.696089,-73.978034
327,E 43 St & 2 Ave,40.750224,-73.971214
328,Grand St & Greene St,40.721655,-74.002347


In [7]:
result1 = locali.to_dict(orient="records")
information.insert_many(result1)
result1 = None
locali = None

### Tabla rutas

In [8]:
information = mydb.tabla_rutas

In [9]:
results = []
for idx, df in enumerate(all_data):
    result = df.to_dict(orient="records")
    information.insert_many(result)
    df = None
    all_data[idx] = None
    result = None

# Actividad 1

La actividad correspondiente al sistema de clientes está en formato .py, ya que involucra una interfaz de usuario. 
Se puede encontrar en el siguiente enlace.

[https://github.com/Axleonr/ClaseBDNE-2022-II/blob/main/P3-Mongo/defs.py](https://github.com/Axleonr/ClaseBDNE-2022-II/blob/main/P3-Mongo/defs.py)

# Actividad 2

In [7]:
from heapq import nsmallest
def estcerc(iniciolon,iniciolat):
    x1=float(iniciolat)
    y1=float(iniciolon)
    b=np.array((y1,x1))
    disttot=[]

    consulta = []
    for j in table2.find():
        consulta.append(j)

    locali2 = pd.DataFrame.from_dict(consulta)

    for i in range(len(locali2['start station longitude'])):
        np.array((locali2['start station latitude'][i],locali2['start station longitude'][i]))
        dist=np.linalg.norm(np.array((locali2['start station latitude'][i],locali2['start station longitude'][i]))-b)
        disttot.append(dist)
    dic={locali2['start station name'][i]:disttot[i] for i in range(len(locali2['start station longitude']))}
    lista=dic.items()
    vad=nsmallest(3, lista)

    print('Las 3 estaciones  más cercanas de tu ubicación son:',vad)

estcerc(40.73535398,-74.00483091)

Las 3 estaciones  más cercanas de tu ubicación son: [('1 Ave & E 15 St', 0.023386479658509103), ('1 Ave & E 18 St', 0.024616963809785602), ('1 Ave & E 30 St', 0.030092739403709417)]


# Actividad 3,4 y Extras

In [8]:
def viaje(tiempo_inicio, lugar_inicio, tiempo_de_viaje,redondo):
    """
    tiempo_inicio: String que indica el dia y hora en el que se planea hacer el viaje
    usuario_id: String id del usuario en la base de datos
    lugar_inicio: String nombre que pusiste a tu lugar de salida
    tiempo_de_viaje: Int El tiempo aproximado que quieres que dure tu viaje en segundos
    Redondo: Booleano True si el viaje lo quieres redondo y False si no es redondo
    """

    tiempo2 = tiempo_inicio + timedelta(minutes=30)

    consulta = []
    for j in table.find({'tripduration':{'$lt':tiempo_de_viaje},
                        'starttime':{'$gt':tiempo_inicio,
                        '$lt':tiempo2},
                        'start station name':lugar_inicio}):
        consulta.append(j)


    df2 = pd.DataFrame.from_dict(consulta)

    df2 = df2.groupby(["start station id","start station name","end station id","end station name"])["tripduration"].mean().round()
    df2 = pd.DataFrame(df2).rename(columns={"tripduration":"Tiempo Estimado"}).reset_index()

    if not redondo:
        df2 = df2.loc[(df2["Tiempo Estimado"] <= tiempo_de_viaje)]
    else:
        df2 = df2.loc[(df2["Tiempo Estimado"] <= (tiempo_de_viaje/2))]
        df2["Tiempo Estimado"] = 2*df2["Tiempo Estimado"]

    return df2[["start station name","end station name","Tiempo Estimado"]]

In [19]:
tiempo_inicio = datetime(year=2014, month=2,day=20,hour=12,minute=58,second=17)
tiempo2 = tiempo_inicio + timedelta(minutes=30)
tiempo_de_viaje = 500
lugar_inicio = 'W 17 St & 8 Ave'

In [20]:
viaje(tiempo_inicio,lugar_inicio,tiempo_de_viaje,True)

,start station name,end station name,Tiempo Estimado
0,W 17 St & 8 Ave,W 22 St & 8 Ave,234.0
2,W 17 St & 8 Ave,9 Ave & W 22 St,406.0


In [21]:
viaje(tiempo_inicio,lugar_inicio,tiempo_de_viaje,False)

,start station name,end station name,Tiempo Estimado
0,W 17 St & 8 Ave,W 22 St & 8 Ave,117.0
1,W 17 St & 8 Ave,W 20 St & 11 Ave,347.0
2,W 17 St & 8 Ave,9 Ave & W 22 St,203.0
3,W 17 St & 8 Ave,8 Ave & W 31 St,330.0


In [12]:
def ruta_especifica(tiempo_inicio,lugar_inicio,lugar_final):
    tiempo2 = tiempo_inicio + timedelta(minutes=30)

    consulta = []
    for j in table.find({'starttime':{'$gt':tiempo_inicio,
                        '$lt':tiempo2}}):
        consulta.append(j)
    df2 = pd.DataFrame.from_dict(consulta)
    df2 = df2.groupby(["start station id","start station name","end station id","end station name"])["tripduration"].mean().round()
    df2 = pd.DataFrame(df2).rename(columns={"tripduration":"Tiempo Estimado"}).reset_index()

    G = nx.Graph()
    for j in range(len(df2)):
        G.add_edge(df2.iloc[j,1],
                df2.iloc[j,3],
                tiempo_espera = df2.iloc[j,4])

    p = nx.shortest_path(G, source=lugar_inicio,target=lugar_final, weight='tiempo_espera')
    tiempo_espera = 0
    for j in range(len(p)-1):

        t = df2.loc[((df2['start station name'] == p[j]) & (df2['end station name'] == p[j+1]))].values
        if len(t) == 0:
            t = df2.loc[((df2['start station name'] == p[j+1]) & (df2['end station name'] == p[j]))].values        
        tiempo_espera += t[0][-1]
    print("tu ruta es la siguiente; ",p)
    print("El tiempo de espera estimado es de: ",tiempo_espera)
        

In [13]:
tiempo_inicio = datetime(year=2014,month=2,day=20,hour=12,minute=0,second=0)
tiempo2 = tiempo_inicio + timedelta(minutes=30)
lugar_inicio = 'Cliff St & Fulton St'
lugar_final = 'Lexington Ave & E 24 St'

In [17]:
ruta_especifica(tiempo_inicio,lugar_inicio,lugar_final)

tu ruta es la siguiente;  ['Cliff St & Fulton St', 'Park Pl & Church St', 'Lafayette St & E 8 St', 'St Marks Pl & 1 Ave', 'Lexington Ave & E 24 St']
El tiempo de espera estimado es de:  1512.0
